# Инференс (использование модели)

Этот ноутбук для тестирования обученной модели на видео и камере.


In [ ]:
# Импорт библиотек
import cv2
from ultralytics import YOLO
import matplotlib.pyplot as plt
from IPython.display import display, Image, clear_output
import time

# Классы для отображения
CLASS_NAMES = {0: "helmet (каска)", 1: "vest (жилет)"}
CLASS_COLORS = {0: (0, 165, 255), 1: (0, 255, 255)}  # BGR

print("✅ Библиотеки для инференса загружены")


## Тестирование на изображении

Загрузите изображение для тестирования модели.


In [ ]:
# Загружаем модель
model_path = "models/ppe_model/weights/best.pt"
if Path(model_path).exists():
    model = YOLO(model_path)
    print(f"✅ Модель загружена: {model_path}")
else:
    print(f"❌ Модель не найдена: {model_path}")
    print("Обучите модель в training.ipynb")

# Функция для детекции на изображении
def detect_image(image_path, model, conf_threshold=0.5):
    image = cv2.imread(str(image_path))
    if image is None:
        print(f"❌ Не удалось загрузить изображение: {image_path}")
        return None
    
    results = model(image, conf=conf_threshold, verbose=False)
    
    # Рисуем детекции
    boxes = results[0].boxes
    if boxes is not None:
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            
            cls = int(box.cls[0].cpu().numpy())
            conf = float(box.conf[0].cpu().numpy())
            
            if conf >= conf_threshold:
                color = CLASS_COLORS.get(cls, (255, 255, 255))
                cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
                
                label = f"{CLASS_NAMES.get(cls, 'unknown')}: {conf:.2f}"
                cv2.putText(image, label, (x1, y1-10), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
    
    # Конвертируем BGR в RGB для отображения
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image_rgb

# Тестирование на изображении из val
test_images = list(Path("data/images/val").glob("*.jpg"))
if test_images and 'model' in locals():
    test_img = test_images[0]
    print(f"Тестируем на: {test_img.name}")
    
    result = detect_image(test_img, model)
    if result is not None:
        plt.figure(figsize=(10, 8))
        plt.imshow(result)
        plt.axis('off')
        plt.title(f"Детекция СИЗ: {test_img.name}")
        plt.show()
else:
    print("Загрузите изображение для тестирования или обучите модель")


## Тестирование на видео

Обработка видео файла с сохранением результата.


In [ ]:
def process_video(video_path, model, output_path="output/detected_video.mp4", conf_threshold=0.5):
    \"\"\"
    Обрабатывает видео с детекцией СИЗ.
    \"\"\"
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        print(f"❌ Не удалось открыть видео: {video_path}")
        return
    
    # Параметры видео
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    print(f"Видео: {width}x{height}, {fps} FPS, {total_frames} кадров")
    
    # Видеозапись
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(str(output_path), fourcc, fps, (width, height))
    
    frame_count = 0
    detections_count = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Детекция
        results = model(frame, conf=conf_threshold, verbose=False)
        boxes = results[0].boxes
        
        # Рисуем детекции
        if boxes is not None:
            for box in boxes:
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                
                cls = int(box.cls[0].cpu().numpy())
                conf = float(box.conf[0].cpu().numpy())
                
                if conf >= conf_threshold:
                    color = CLASS_COLORS.get(cls, (255, 255, 255))
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                    
                    label = f"{CLASS_NAMES.get(cls, 'unknown')}: {conf:.2f}"
                    cv2.putText(frame, label, (x1, y1-10), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                    detections_count += 1
        
        # Счетчик кадров
        cv2.putText(frame, f"Frame: {frame_count}", (10, 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        out.write(frame)
        frame_count += 1
        
        if frame_count % 30 == 0:
            progress = frame_count / total_frames * 100
            print(f"Прогресс: {progress:.1f}% ({frame_count}/{total_frames})")
    
    cap.release()
    out.release()
    
    print(f"\n✅ Видео обработано!")
    print(f"Результат: {output_path}")
    print(f"Детекций найдено: {detections_count}")
    return output_path

# Обработка видео (замените путь на ваш файл)
# video_path = "videos/test_video.mp4"  # Укажите путь к видео
# if 'model' in locals() and Path(video_path).exists():
#     result_video = process_video(video_path, model)
#     print(f"Видео с детекцией: {result_video}")
#else:
#    print("Загрузите видео файл или обучите модель")


In [ ]:
# Детекция с камеры (работает только локально)
def detect_camera(model, conf_threshold=0.5, camera_id=0):
    cap = cv2.VideoCapture(camera_id)
    if not cap.isOpened():
        print(f"❌ Не удалось открыть камеру {camera_id}")
        return
    
    print("Камера запущена! Нажмите 'q' для выхода.")
    frame_count = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Детекция
        results = model(frame, conf=conf_threshold, verbose=False)
        boxes = results[0].boxes
        
        # Рисуем детекции
        if boxes is not None:
            for box in boxes:
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                
                cls = int(box.cls[0].cpu().numpy())
                conf = float(box.conf[0].cpu().numpy())
                
                if conf >= conf_threshold:
                    color = CLASS_COLORS.get(cls, (255, 255, 255))
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                    
                    label = f"{CLASS_NAMES.get(cls, 'unknown')}: {conf:.2f}"
                    cv2.putText(frame, label, (x1, y1-10), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        
        # Информация
        cv2.putText(frame, f"Frame: {frame_count}", (10, 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(frame, "Press 'q' to quit", (10, 60), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
        cv2.imshow('PPE Detection - Real Time', frame)
        frame_count += 1
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    print("Камера остановлена.")

# Запуск камеры (раскомментируйте для запуска)
# if 'model' in locals():
#     detect_camera(model)
#else:
#    print("Обучите модель в training.ipynb")
